In [101]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import h5py

In [102]:
fname = "data/ganglion_cell_data/15-11-21a/naturalscene.h5"
f = h5py.File(fname,"r")

f["train"]["stimulus"]

<HDF5 dataset "stimulus": shape (359802, 50, 50), type "|u1">